# Core hollowing in the early MEBT

We see longitudinal and, to some extend, transverse hollowing in our measurements at slit HZ04, 1.4 meters downstream of the RFQ. This notebook explores the space charge dependence of these features using RFQ simulations. The simulations were performed using the PARMTEQ code with approximately 9 million macroparticles. 

In [ ]:
import sys
import os
from os.path import join
from pprint import pprint
from tqdm.notebook import tqdm
from tqdm.notebook import trange
import importlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import proplot as pplt
from ipywidgets import widgets
from ipywidgets import interactive

sys.path.append('/Users/46h/Research/')
from beamphys import plotting as mplt
from beamphys import utils

pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['figure.facecolor'] = 'white'
pplt.rc['savefig.dpi'] = 'figure'

# pplt.rc['pdf.fonttype'] = 42

## Load data

In [ ]:
beam_currents = [0, 22, 26, 42, 50]
beam_currents_str = [f'{c}mA' for c in beam_currents]

In [ ]:
folder = '/Users/46h/Dropbox (ORNL)/work/btf/simulations/2022-09-20/'
filenames = dict()
for current in beam_currents_str:
    _folder = join(folder, current)
    _filenames = os.listdir(_folder)
    filenames[current] = dict()
    for _filename in _filenames:
        # print(_filename)
        if _filename.endswith('.py'):
            continue
        if _filename.split('.dat')[0].endswith('history'):
            filenames[current]['history'] = join(_folder, _filename)
        elif 'bunch' in _filename:
            node = _filename.split('.dat')[0].split('bunch_')[-1]
            if node.startswith('MEBT:'):
                node = node.split('MEBT:')[-1]
            filenames[current][node] = join(_folder, _filename)
        else:
            pass
            
print('filenames:')
pprint(filenames)

## RMS 

In [ ]:
def load_history(filename):
    df = pd.read_table(filename, sep=' ')
    # # Convert rms x and y emittance from m-rad to mm-mrad
    # df[['eps_x', 'eps_y']] *= 1e3 * 1e3  
    # # Convert rms z emittance from m-GeV to mm-keV
    # df[['eps_z']] *= 1e3 * 1e6
    return df

In [ ]:
figwidth = 8.0

In [ ]:
fig, axes = pplt.subplots(ncols=len(filenames), figwidth=figwidth)
for ax, current in zip(axes, beam_currents_str):
    history = load_history(filenames[current]['history'])
    for dim in ['x', 'y', 'z']:
        data = history[f'eps_{dim}'].values
        ax.plot(history['s'], data / data[0], label=r'$\varepsilon_{}$'.format(dim), marker='.', ms=1, lw=0, alpha=1.0)
axes[0].legend(ncol=1, loc='upper left', ms=3)
axes.format(xlabel='Distance [m]', ylabel='Relative growth', toplabels=beam_currents_str)
plt.savefig('_output/relative_rms_emittance_growth.png')
plt.show()

In [ ]:
print(sorted(history.columns))

In [ ]:
fig, axes = pplt.subplots(ncols=len(filenames), figwidth=figwidth)
for ax, current in zip(axes, beam_currents_str):
    history = load_history(filenames[current]['history'])
    for i, dim in enumerate(['x', 'y', 'z']):
        i *= 2
        col = f'sig_{i}{i}'
        data = np.sqrt(history[col].values)
        data *= 1e3  # convert [m] to [mm]
        ax.plot(history['s'], data, label=r'${}$'.format(dim))
axes[0].legend(ncol=1, loc='upper left')
axes.format(xlabel='Distance [m]', ylabel='RMS beam size [mm]', toplabels=beam_currents_str)
plt.savefig('_output/rms_beam_size.png')
plt.show()

## Distribution 

In [ ]:
def load_bunch(filename, dims=None, dframe=False):
    names = ["x", "xp", "y", "yp", "z", "dE"]
    cols = list(range(6))
    if dims is not None:
        cols = [d if type(d) is int else names.index(d) for d in dims]
    names = [names[c] for c in cols]
    df = pd.read_table(filename, sep=' ', skiprows=14, usecols=cols, names=names)
    # Convert to mm, mrad, keV
    for col in ['x', 'xp', 'y', 'yp', 'z']:
        if col in df.columns:
            df[col] *= 1e3
    if 'dE' in df.columns:
        df['dE'] *= 1e6
    if dframe:
        return df
    return df.values

In [ ]:
dims = ["x", "x'", "y", "y'", "z", "dE"]
units = ["mm", "mrad", "mm", "mrad", "mm", "keV"]
labels = [f'{d} [{u}]' for d, u in zip(dims, units)]

In [ ]:
nodes = ['init', 'QH01', 'QV02', 'QH03', 'QV04', 'HZ04']

In [ ]:
plot_kws = dict(
    prof_kws=dict(lw=0.8, alpha=0.4, color='white'),
    mask_zero=False,
    # cmap=pplt.Colormap('mono', left=0.03, right=0.85),
)
nbins = 33

In [ ]:
node = 'QV02'

In [ ]:
mplt.interactive_proj2d_discrete(
    load_bunch(filenames['0mA'][node]),
    dims=dims,
    units=units,
    nbins=nbins,
    **plot_kws
)

In [ ]:
mplt.interactive_proj2d_discrete(
    load_bunch(filenames['22mA'][node]),
    dims=dims,
    units=units,
    nbins=nbins,
    **plot_kws
)

In [ ]:
mplt.interactive_proj2d_discrete(
    load_bunch(filenames['26mA'][node]),
    dims=dims,
    units=units,
    nbins=nbins,
    **plot_kws
)

In [ ]:
mplt.interactive_proj2d_discrete(
    load_bunch(filenames['42mA'][node]),
    dims=dims,
    units=units,
    nbins=nbins,
    **plot_kws
)

In [ ]:
mplt.interactive_proj2d_discrete(
    load_bunch(filenames['50mA'][node]),
    dims=dims,
    units=units,
    nbins=nbins,
    **plot_kws
)

Eh, why not load everything into memory. Can change if it's too much.

In [ ]:
coords = dict()
for current in tqdm(beam_currents_str):
    coords[current] = dict()
    for node in tqdm(nodes):
        X = load_bunch(filenames[beam_current][node])
        coords[current][node] = X.copy()

Compute limits from coordinates.

In [ ]:
current = '26mA'
pairs = [(0, 2), (0, 1), (2, 3), (4, 5)]

limits = mplt.auto_limits(
    np.vstack([
        coords[current][node] 
        for current in beam_currents_str
        for node in nodes
    ]),
    zero_center=True,
)

fig, axes = pplt.subplots(ncols=len(nodes), nrows=len(pairs), figwidth=9, share=False, space=0)
axes.format(toplabels=nodes, leftlabels=[f'{dims[p[0]]}-{dims[p[1]]}' for p in pairs],
            leftlabels_kw=dict(rotation=0))
for j, node in enumerate(tqdm(nodes[:])):
    X = coords[current][node]
    for i, cols in enumerate(tqdm(pairs)):
        _X = X[:, cols]
        sns.histplot(
            x=_X[:, 0], y=_X[:, 1], ax=axes[i, j], bins='auto', 
            # binrange=(limits[cols[0]], limits[cols[1]])
        )
for i, cols in enumerate(pairs):
    axes[i, :].format(xlim=limits[cols[0]], ylim=limits[cols[1]])
axes.format(xticks=[], yticks=[])
plt.show()

### Interactive

In [ ]:
mplt.interactive_proj2d_discrete(
    coords['0mA']['HZ04'],
    # limits=None,
    slice_type="int",
    dims=dims,
    units=units,
)

In [ ]:
axes = mplt.corner(
    X, labels=labels, kind='hist',
    mask_zero=True,
)
plt.savefig('_output/corner.png')

5D distribution (beware large grids)

In [ ]:
ind = (0, 1, 2, 3, 5)
_dims = [dims[i] for i in ind]
_units = [units[i] for i in ind]
_labels = [labels[i] for i in ind]

f, edges = np.histogramdd(X[:, ind], bins=34)
f = f / np.max(f)

_coords = [utils.get_bin_centers(e) for e in edges]

In [ ]:
mplt.interactive_proj2d(f, coords=_coords, dims=_dims, units=_units,
                        slice_type='int')

In [ ]:
axis = (0, 2, 4)
_f = utils.project(f, axis=axis)
_f = _f / np.max(_f)
__coords = [_coords[i] for i in axis]
__dims = [_dims[i] for i in axis]
__dims[-1] = 'w'
__units = [_units[i] for i in axis]
__dims_units = [f'{d} [{u}]' for d, u in zip(__dims, __units)]

ncols = 5
start_stop = [
    (4, len(__coords[0]) - 1 - 4),
    (10, len(__coords[1]) - 1 - 10),
    (25, len(__coords[2]) - 1 - 25),
]

cmap = 'viridis'
for i, ((start, stop), c) in enumerate(zip(start_stop, __coords)):
    fig, axes = pplt.subplots(ncols=ncols, nrows=1, figwidth=6.0)
    jj = np.linspace(start, stop, ncols).astype(int)
    for ax, j in zip(axes, jj):
        idx = utils.make_slice(_f.ndim, i, int(j))
        axis_view = [k for k in range(_f.ndim) if k != i]
        mplt.plot_image(
            _f[idx], 
            x=__coords[axis_view[0]], 
            y=__coords[axis_view[1]], 
            ax=ax,
            vmin=0.0, vmax=1.0,
            linewidth=0, rasterized=True,
            cmap=cmap,
            # colorbar=ax==axes[-1], 
            colorbar_kw=dict(
                width=0.1, 
                # ticks=[0, 1],
            ),
            profx=True, profy=True, prof_kws=dict(lw=0.4, alpha=0.5, color='white', scale=0.09),
        )
        if ax.get_xlim()[1] < ax.get_xlim()[0]:
            ax.format(xlim=sorted(ax.get_xlim()))
        ax.annotate(r"${} \approx {:.0f}$ [{}]".format(__dims[i], __coords[i][j], __units[i]), 
                    color='white', 
                    fontsize='small',
                    xy=(0.02, 0.98), verticalalignment='top', xycoords='axes fraction')
    axes.format(
        xlabel=__dims_units[axis_view[0]], 
        ylabel=__dims_units[axis_view[1]],
    )
    plt.savefig(f'_output/{__dims[axis_view[0]]}{__dims[axis_view[1]]}_{__dims[i]}slice_{cmap}')
    plt.show()